In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import cv2
import matplotlib.pyplot as plt
import os
import seaborn as sns
import umap
from PIL import Image
from scipy import misc
from os import listdir
from os.path import isfile, join
import numpy as np
from scipy import misc
from random import shuffle
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical


Using TensorFlow backend.


We can split the data into Classes
* Children (1-14) CLASS 0
* Youth (14-25) CLASS 1
*  ADULTS (25-40) CLASS 2
*  Middle age (40-60) CLASS 3
*  Very Old (>60) CLASS 4

**CONVERT IMAGES TO VECTORS**

In [2]:
os.chdir('/kaggle/input/UTKFace/')
temp = os.listdir()
import fnmatch
dataset = fnmatch.filter(temp, '*_*_*_*')
m = len(dataset)
print(m)
shuffle(dataset)
age = [int(i.split('_')[0])/10 for i in dataset]
age = to_categorical(age,num_classes=12)
X_data =[]
for file in dataset:
    face = cv2.imread(file)
    face = cv2.resize(face, (128, 128) )
    X_data.append(face)
X = np.squeeze(X_data)

23705


In [3]:
X = np.squeeze(X_data)

In [4]:
X.shape

(23705, 128, 128, 3)

In [5]:
# normalize data
X = X.astype('float32')
X /= 255


In [6]:
X_train=X[:7000]
X_validate=X[7000:8500]
X_test=X[8500:]
age = np.squeeze(age)
Y_train=age[:7000]
Y_validate=age[7000:8500]
Y_test=age[8500:]

In [7]:
len(X_train)+len(X_test) + len(X_validate) == len(X)


True

In [8]:
import keras
model=Sequential()
model.add(Conv2D(64, (7, 7), padding="same", activation="relu", input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (5, 5), padding="same", strides =(1, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), padding="same", strides =(2, 2), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(526, activation ='relu')) 
model.add(Dense(12, activation ='softmax')) 

model.compile(loss = keras.losses.categorical_crossentropy, 
              optimizer = keras.optimizers.SGD(lr = 0.01), 
              metrics =['accuracy']) 

# Take a look at the model summary
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 64)      9472      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        51232     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
__________

In [9]:
model.fit(X_train,
         Y_train,
         batch_size=64,
         epochs=50,
         validation_data=(X_validate, Y_validate),)

Train on 7000 samples, validate on 1500 samples
Epoch 1/50
7000/7000 [==============================] - 11s 2ms/step - loss: 2.0146 - acc: 0.2863 - val_loss: 1.9568 - val_acc: 0.2340
Epoch 2/50
7000/7000 [==============================] - 5s 663us/step - loss: 1.9119 - acc: 0.3124 - val_loss: 1.9620 - val_acc: 0.2733
Epoch 3/50
7000/7000 [==============================] - 5s 670us/step - loss: 1.8653 - acc: 0.3321 - val_loss: 2.0621 - val_acc: 0.1607
Epoch 4/50
7000/7000 [==============================] - 5s 667us/step - loss: 1.8298 - acc: 0.3434 - val_loss: 1.8562 - val_acc: 0.3140
Epoch 5/50
7000/7000 [==============================] - 5s 668us/step - loss: 1.7863 - acc: 0.3617 - val_loss: 1.7488 - val_acc: 0.3680
Epoch 6/50
7000/7000 [==============================] - 5s 673us/step - loss: 1.7108 - acc: 0.3883 - val_loss: 1.8929 - val_acc: 0.3707
Epoch 7/50
7000/7000 [==============================] - 5s 672us/step - loss: 1.6614 - acc: 0.4067 - val_loss: 1.7565 - val_acc: 0.3807
E

In [10]:
# Evaluate the model on test set
score = model.evaluate(X_test, Y_test, verbose=0)

# Print test accuracy
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.4609010194093528
